In [1]:
%pwd

'c:\\Users\\sajit\\OneDrive\\Documents\\Desktop\\Pythonn\\Git\\Named-Entity-Recognition-News\\backend\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\sajit\\OneDrive\\Documents\\Desktop\\Pythonn\\Git\\Named-Entity-Recognition-News\\backend'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TokenizerPreparationConfig:
    root_dir: Path
    data_path: Path


In [5]:
from named_entity_recognition.constants import *
from named_entity_recognition.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_tokenizer_preparation_config(self) -> TokenizerPreparationConfig:

        config = self.config.tokenizer_preparation

        create_directories([config.root_dir])

        tokenizer_preparation_config = TokenizerPreparationConfig(
            root_dir = Path(config.root_dir),
            data_path = Path(config.data_path)
        )

        return tokenizer_preparation_config    
        

In [20]:
import os
import pickle
from datasets import load_from_disk
from named_entity_recognition import logger
from named_entity_recognition.utils.common import get_size
import tensorflow as tf

In [24]:
class TokenizerPreparation:

    def __init__(self, config: TokenizerPreparationConfig):
        self.config = config

    def prepare_tokenizer(self):
        data = load_from_disk(self.config.data_path)
        tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token="[UNK]", lower=True)
        tokenizer.fit_on_texts([' '.join(x) for x in data['train']['tokens']])
        with open(os.path.join(self.config.root_dir, 'tokenizer.pickle'), 'wb') as f:
            pickle.dump(tokenizer, f)
        logger.info(f"Tokenizer created successfully at {self.config.root_dir}")

In [26]:
try:
    config = ConfigurationManager()
    tokenizer_preparation_config = config.get_tokenizer_preparation_config()
    tokenizer_preparation = TokenizerPreparation(config=tokenizer_preparation_config)
    tokenizer_preparation.prepare_tokenizer()
except Exception as e:
    raise e

[2024-04-22 23:01:46,174: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-22 23:01:46,176: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-22 23:01:46,177: INFO: common: created directory at: artifacts]
[2024-04-22 23:01:46,178: INFO: common: created directory at: artifacts/tokenizer_preparation]
[2024-04-22 23:01:46,527: INFO: 915053661: Tokenizer created successfully at artifacts\tokenizer_preparation]
